<a href="https://colab.research.google.com/github/kazemnejad/tensorflow-2-tutorial/blob/master/part_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow 2.0 Tutorial: Part #1


Deep Learning Group, Iran University of Science and Technology,

*   Last Update: Dec 2019
*   Official Page: https://github.com/iust-deep-learning/tensorflow-2-tutorial





---




Please run the following cell before going through the rest of the tutorial.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np

from pprint import pprint

## 1. Computation Graph

---



# What is Computational Graph?


# TensorFlow behind the scenes

## 2. Tensors, Variables, and OPS

---



### Tensors
Tensors are the main element that you will use to define your desired computations. Generally speaking, Tensors are n-dimensional arrays with a specified data type. That is, each component of the Tensors has the same data type (e.g., int32 or float32), and such a data type is always known across the computation. Various methods can create tensors, two of which–that is–**constants and variables** are the most common ones.

*   **Constants**

Use methods such as `tf.ones(...)`, `tf.zeros(...)`, `tf.eye(...)`, and etc..




In [0]:
a = tf.ones(shape=(2,3), dtype=tf.int32)
a

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1]], dtype=int32)>

Or define them by manually passing Python/numpy data types ([More info](https://www.tensorflow.org/api_docs/python/tf/constant))



In [0]:
b = tf.constant([[1, 2, 3], [4, 5, 6]])
print("b =",b);

npvar = np.array(["hello", "world"])
c = tf.constant(npvar)
print("\nc =", c)

d = tf.constant(10.0, shape=[2,5])
print("\nd =", d)

b = tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)

c = tf.Tensor([b'hello' b'world'], shape=(2,), dtype=string)

d = tf.Tensor(
[[10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]], shape=(2, 5), dtype=float32)


You can also use random initializers ([More info](https://www.tensorflow.org/api_docs/python/tf/random)). You may re-run the cell to generate another set of random values.

In [0]:
e = tf.random.normal(shape=[2, 3], mean=0.0, stddev=1.0)
print("e =", e)

f = tf.random.uniform( shape=[2,3], minval=0,maxval=10,dtype=tf.int32)
print("\nf =", f)

e = tf.Tensor(
[[ 0.61113954 -0.258195    1.2110202 ]
 [-0.39822677 -2.3552241  -3.0503285 ]], shape=(2, 3), dtype=float32)

f = tf.Tensor(
[[3 3 0]
 [3 4 7]], shape=(2, 3), dtype=int32)


*   **Variables**

Variables hold a persistant shared state across your computation. The most common use case of Variables is the model's trainable parameters.

The only way to create variables is to use `tf.Variable(<required-initial-value>, name=<optional-name>)` class. Tensorflow uses the `initial-value` to infer the shape and the type of the variable. Please note that shape and the type of variable, once specified, cannot be changed during the computation.  Tensorflow cleans up variables when the runtime changes its scope and the variable is not referenced anymore. Therefore, it is your responsibility to keep track of variables in your Tensorflow program (Good news: Tensorflow's high-level APIs handles that automatically)

In [0]:
w = tf.Variable(20., name="my_var01")
print('w =', w)

initializer = tf.initializers.GlorotUniform()
x = tf.Variable(initializer(shape=(2, 5)), name="my_var02")
print('\nx =', x)

y = tf.Variable(tf.zeros([5]), name='my_var03')
print('\ny =', y)

w = <tf.Variable 'my_var01:0' shape=() dtype=float32, numpy=20.0>

x = <tf.Variable 'my_var02:0' shape=(2, 5) dtype=float32, numpy=
array([[-0.45070884,  0.56897163,  0.29169297, -0.77587044, -0.4456739 ],
       [ 0.1657275 , -0.9256539 , -0.86812764,  0.05088377,  0.19746172]],
      dtype=float32)>

y = <tf.Variable 'my_var03:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>


Variables' APIs are mostly similar to Tensors. Hence, we can treat them like a standard Tensor.

In [0]:
v = w + 1.  # v is a tf.Tensor and is calculated as the result of
            # a mathematical expression that is based on a variable(w).
            # tf.Variable gets automatically converted to a tf.Tensor 
            # representing its value when it is envolved in a expression.

print("v =", v)
print(f"v's type = {type(v)}")
print(f"w's type = {type(w)}")

v = tf.Tensor(21.0, shape=(), dtype=float32)
v's type = <class 'tensorflow.python.framework.ops.EagerTensor'>
w's type = <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>


To change the variable's current value, you can use methods such as `assign` and `assign_add`. ([More info](https://www.tensorflow.org/api_docs/python/tf/Variable))

In [0]:
w.assign(v)
w.assign_add(v)
print('w =', w)

w = <tf.Variable 'my_var01:0' shape=() dtype=float32, numpy=42.0>


### Rank, Shape, and Type Conversion

In [0]:
print(f"a = \n{a}")
print("a.dtype =", a.dtype)
print("a.shape =", a.shape)
print("a.rank =", len(a.shape))
# or...
print("\na.shape =", tf.shape(a))
print("a.rank =", tf.rank(a)) 
# What is the difference?

print("\ne (before type conversion) =", e)
e_int = tf.cast(e, tf.int32)
print("e (after type conversion) =", e_int)

# Convert a tf.Tensor object to an np.array instance
e_np = e_int.numpy()
print(f"\ntype(e_np) = {type(e_np)}")
e_np

a = 
[[1 1 1]
 [1 1 1]]
a.dtype = <dtype: 'int32'>
a.shape = (2, 3)
a.rank = 2

a.shape = tf.Tensor([2 3], shape=(2,), dtype=int32)
a.rank = tf.Tensor(2, shape=(), dtype=int32)

e (before type conversion) = tf.Tensor(
[[ 0.61113954 -0.258195    1.2110202 ]
 [-0.39822677 -2.3552241  -3.0503285 ]], shape=(2, 3), dtype=float32)
e (after type conversion) = tf.Tensor(
[[ 0  0  1]
 [ 0 -2 -3]], shape=(2, 3), dtype=int32)

type(e_np) = <class 'numpy.ndarray'>


array([[ 0,  0,  1],
       [ 0, -2, -3]], dtype=int32)

### Tensor manipulation 

*   **Element-wise operations**

In [0]:
t1 = tf.constant([[0, 0, 0], [0, 1, 1], [0, 1, 1]])
t2 = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print('t1 + t2 =', t1 + t2)
print('t2 - t1 =', t2 - t1)
print('t1 * t2 =', t1 * t2)
print('t1 / t2 =', t1 / t2)

t1 + t2 = tf.Tensor(
[[ 1  2  3]
 [ 4  6  7]
 [ 7  9 10]], shape=(3, 3), dtype=int32)
t2 - t1 = tf.Tensor(
[[1 2 3]
 [4 4 5]
 [7 7 8]], shape=(3, 3), dtype=int32)
t1 * t2 = tf.Tensor(
[[0 0 0]
 [0 5 6]
 [0 8 9]], shape=(3, 3), dtype=int32)
t1 / t2 = tf.Tensor(
[[0.         0.         0.        ]
 [0.         0.2        0.16666667]
 [0.         0.125      0.11111111]], shape=(3, 3), dtype=float64)


*   **Broadcasting**

<p align="center">
<img src="https://raw.githubusercontent.com/kazemnejad/tensorflow-2-tutorial/master/resources/part_01_broadcasting.jpg" width="500" />

<a href="https://www.tutorialspoint.com/numpy/numpy_broadcasting.htm">[source]</a>
</p>

Broadcasting happens in arithmetic operations encountering tensors with different shapes. Basically, Tensorflow "broadcasts" the smaller tensor across the larger matrix so that they become compatible. Think of broadcasting as repeating the values of the smaller tensor without actually needlessly copying them. In fact, Broadcasting provides an easy way to implement algorithms efficiently.

In [84]:
t1 = tf.constant([1, 2, 3, 4])
print("t1 + 100 =", t1 + 100)

# (m, n) + (1, n)
t1 = tf.constant([[1, 2, 3], 
                  [4, 5, 6]])
t2 = tf.constant([[100, 200, 300]])
print(f"\nt1.shape = {t1.shape}, t2.shape = {t2.shape}")
print("t1 + t2 =", t1 + t2)

# (m, n) + (n, 1)
t1 = tf.constant([[1, 2, 3], 
                  [4, 5, 6]])
t2 = tf.constant([[100], 
                  [200]])
print(f"\nt1.shape = {t1.shape}, t2.shape = {t2.shape}")
print("t1 + t2 =", t1 + t2)

# (1, n) + (m, 1)
t1 = tf.constant([[1, 2, 3]])
t2 = tf.constant([[100], 
                  [200]])
print(f"\nt1.shape = {t1.shape}, t2.shape = {t2.shape}")
print("t1 + t2 =", t1 + t2)

t1 + 100 = tf.Tensor([101 102 103 104], shape=(4,), dtype=int32)

t1.shape = (2, 3), t2.shape = (1, 3)
t1 + t2 = tf.Tensor(
[[101 202 303]
 [104 205 306]], shape=(2, 3), dtype=int32)

t1.shape = (2, 3), t2.shape = (2, 1)
t1 + t2 = tf.Tensor(
[[101 102 103]
 [204 205 206]], shape=(2, 3), dtype=int32)

t1.shape = (1, 3), t2.shape = (2, 1)
t1 + t2 = tf.Tensor(
[[101 102 103]
 [201 202 203]], shape=(2, 3), dtype=int32)


In [0]:
# General Rule

# 1.  (m, n) matrix    +, -, *, /    (1, n) matrix   =(get copied)=>   (m, n)
# 2.  (m, n) matrix    +, -, *, /    (m, 1) matrix   =(get copied)=>   (m, n)
# 2.  (m, n) matrix    +, -, *, /    0D scalar       =(get copied)=>   (m, n)

## 3. Auto Differentiation

---



# Assignment!

---



## References